In [ ]:
import pickle
import os

import AE as ae

import numpy as np

import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

## Load Data

In [ ]:
directory = 'INSERT DIRECTORY' # Directory where events generated by Simulation.py are saved.

file = 'INSERT FILENAME' # Filename of generated data

snr = float(file.split('_')[0])

# Load Data and Ground Truths
raw_traces, ground_truths = ae.load_SNR_data(directory, [file])

# Load background analysis data
stat_dir = 'INSERT DIRECTORY' # Directory where background analysis results generated by AE Background.ipynb are saved.
with open(os.path.join(stat_dir, 'INSERT FILENAME'), 'rb') as f:
    background_mean, background_std = pickle.load(f)

In [ ]:
### Constants
window_size = 5 # Must match network input size
step_size = 5

model_dir = 'INSERT FOLDER NAME' # Path to network model save location

In [ ]:
# Load pretrained AE model
model = tf.keras.models.load_model(model_dir)
model.summary()

## Analyse Background

In [ ]:
# Load precomputed background analysis results
with open(os.path.join(stat_dir, 'mae_thresholds.pkl'), 'rb') as f:
    mae_thresholds = pickle.load(f)

In [ ]:
mae_thresholds

## Generate MAE Losses

In [ ]:
traces = raw_traces[snr]
traces_norm = (traces - background_mean) / background_std
traces_maes = np.array(ae.calculate_maes(traces_norm, model, window_size, step_size))
traces_maes = traces_maes.reshape(traces_maes.shape[:-1])
traces_maes.shape

# Save MAEs

In [ ]:
fname = '{}_maes.pkl'.format(snr)
with open(fname, 'wb') as f:
    pickle.dump(traces_maes, f)